# Imports

In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable, Expression
from biogeme.models import loglogit
from biogeme.tools import likelihood_ratio_test
from biogeme.results import compile_estimation_results
from biogeme.models import loglogit,  boxcox
from biogeme.models.piecewise import piecewise_formula
from biogeme.models import lognested
from biogeme.nests import OneNestForNestedLogit, NestsForNestedLogit
from biogeme.biogeme import BIOGEME

import pickle

import numpy as np
import os

from scipy.stats import chi2

# Data & Variables

In [2]:
# Define the relative path to the data folder
file_path = os.path.join(os.pardir, 'lpmc01.dat')

#file_path = os.path.join(data_folder, 'lpmc01.dat')

df = pd.read_csv(file_path, sep = '\t')
df['age_normalized'] = (df['age'] - df['age'].mean()) / df['age'].std()
df['age_scaled'] = df['age'] / df['age'].max()
df['cost_driving'] = df['cost_driving_ccharge'] + df['cost_driving_fuel']
df['dur_pt'] = df['dur_pt_access'] + df['dur_pt_rail'] + df['dur_pt_int'] + df['dur_pt_bus']

columns_to_remove = ['cost_driving_ccharge', 'cost_driving_fuel', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_int', 'dur_pt_bus']
df = df.drop(columns=columns_to_remove)


database1 = db.Database('lpmc01', df)


# Define the given veriables 
dur_pt = Variable('dur_pt')
cost_driving = Variable('cost_driving')
age_scaled = Variable('age_scaled')
trip_id = Variable('trip_id')
household_id = Variable('household_id')
person_n = Variable('person_n')
trip_n = Variable('trip_n')
travel_mode = Variable('travel_mode')
purpose = Variable('purpose')
fueltype = Variable('fueltype')
faretype = Variable('faretype')
bus_scale = Variable('bus_scale')
survey_year = Variable('survey_year')
travel_year = Variable('travel_year')
travel_month = Variable('travel_month')
travel_date = Variable('travel_date')
day_of_week = Variable('day_of_week')
start_time = Variable('start_time')
age = Variable('age')
female = Variable('female')
driving_license = Variable('driving_license')
car_ownership = Variable('car_ownership')
distance = Variable('distance')
dur_walking = Variable('dur_walking')
dur_cycling = Variable('dur_cycling')
pt_interchanges = Variable('pt_interchanges')   # Number of interchange points in public transport route
dur_driving = Variable('dur_driving')
cost_transit = Variable('cost_transit')
driving_traffic_percent = Variable('driving_traffic_percent')


variable_names = ['dur_pt', 'cost_driving', 'age_scaled']  # Replace with your variable name
for variable_name in variable_names:
    if variable_name in database1.data.columns:
        print(f"'{variable_name}' exists in the database.")
    else:
        print(f"'{variable_name}' does NOT exist in the database.")



# Define pt_cost (not needed)
# Original paper, page 31: "Public transport fares are determined for single trips using Oystercard/contactless payment."
# Therefore, cost_transit should already consider faretype and bus_scale

database = db.Database('lpmc01', df)
variable_names = ['dur_pt', 'cost_driving', 'age_scaled']  # Replace with your variable name
for variable_name in variable_names:
    if variable_name in database1.data.columns:
        print(f"'{variable_name}' exists in the database.")
    else:
        print(f"'{variable_name}' does NOT exist in the database.")

'dur_pt' exists in the database.
'cost_driving' exists in the database.
'age_scaled' exists in the database.
'dur_pt' exists in the database.
'cost_driving' exists in the database.
'age_scaled' exists in the database.


# Model Definition

In [3]:
# Assume every mode of transport is available
availability_walk = 1  
availability_cycle = 1  
availability_pt = 1     
availability_drive = 1

availability = {
    1: availability_walk,   # Walking
    2: availability_cycle,  # Cycling
    3: availability_pt,     # Public Transport
    4: availability_drive   # Driving
}

In [4]:
thresholds = [0, 2, 5, None]

ALPHA_0_1 = Beta('ALPHA_0_1', 0, None, None, 0)
ALPHA_1_5 = Beta('ALPHA_1_5', 0, None, None, 0)
ALPHA_5_INF = Beta('ALPHA_5_INF', 0, None, None, 0)
betas_piecewise = [ALPHA_0_1, 
                   ALPHA_1_5, 
                   ALPHA_5_INF]

piecewise_cost_driving = piecewise_formula(
    'cost_driving', thresholds, betas_piecewise
)

piecewise_cost_pt = piecewise_formula(
    'cost_transit', thresholds, betas_piecewise
)

In [5]:
# Define alternative-specific parameters for travel time
B_TIME_WALK = Beta('B_TIME_WALK', 0, None, None, 0)
B_TIME_CYCLE = Beta('B_TIME_CYCLE', 0, None, None, 0)
B_TIME_PT = Beta('B_TIME_PT', 0, None, None, 0)
B_TIME_DRIVE = Beta('B_TIME_DRIVE', 0, None, None, 0)

ASC_CYCLE = Beta('ASC_CYCLE', 0, None, None, 0)
ASC_PT = Beta('ASC_PT', 0, None, None, 0)
ASC_DRIVE = Beta('ASC_DRIVE', 0, None, None, 0)

# Define generic parameters for cost and travel time
B_COST = Beta('B_COST', 0, None, None, 0)

G_TIME_WALK_AGE = Beta('G_TIME_WALK_AGE', 0, None, None, 0)
G_TIME_CYCLE_AGE = Beta('G_TIME_CYCLE_AGE', 0, None, None, 0) 
G_TIME_PT_AGE = Beta('G_TIME_PT_AGE', 0, None, None, 0)
G_TIME_DRIVE_AGE = Beta('G_TIME_DRIVE_AGE', 0, None, None, 0) 

# Utility functions with interaction terms
V_WALK = (B_TIME_WALK + G_TIME_WALK_AGE * age_scaled) * dur_walking
V_CYCLE = ASC_CYCLE + (B_TIME_CYCLE + G_TIME_CYCLE_AGE* age_scaled) * dur_cycling
V_PT = ASC_PT + piecewise_cost_pt + (B_TIME_PT + G_TIME_PT_AGE* age_scaled) * dur_pt
V_DRIVE = ASC_DRIVE +  piecewise_cost_driving + (B_TIME_DRIVE + G_TIME_DRIVE_AGE * age_scaled) * dur_driving

# Associate utility functions with the mode choice
V = {
    1: V_WALK,    # Walking
    2: V_CYCLE,   # Cycling
    3: V_PT,      # Public Transport
    4: V_DRIVE    # Driving
}

logprob_m4 = loglogit(V, availability, travel_mode)

model_4 = bio.BIOGEME(database, logprob_m4)
model_4.modelName = 'model_4'

results_m4 = model_4.estimate()
model_4_loglike = results_m4.data.logLike
model_4_numParam = results_m4.get_estimated_parameters().shape[0]
model_4_table = results_m4.get_estimated_parameters()

model_4_table

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ALPHA_0_1,-0.448813,0.049260,-9.111123,0.000000e+00
ALPHA_1_5,0.057885,0.074087,0.781313,4.346184e-01
ALPHA_5_INF,-0.215778,0.041184,-5.239374,1.611224e-07
ASC_CYCLE,-4.664685,0.204420,-22.819128,0.000000e+00
ASC_DRIVE,-2.162054,0.150816,-14.335693,0.000000e+00
ASC_PT,-2.382540,0.154743,-15.396740,0.000000e+00
B_TIME_CYCLE,-4.945503,0.899390,-5.498727,3.825422e-08
B_TIME_DRIVE,-5.704884,0.901937,-6.325145,2.529945e-10
B_TIME_PT,-2.589935,0.493343,-5.249765,1.522933e-07
B_TIME_WALK,-7.617824,0.513991,-14.820941,0.000000e+00


# Market Share Stuff

In [6]:
data_filtered = df

populations = {
    'female_45_less': 2841376,
    'female_45_or_more': 1519948,
    'male_45_less': 2929408,
    'male_45_or_more': 1379198,
}

total_pop = sum(populations.values())

filters = {
    'male_45_or_more': (data_filtered.age >= 45) & (data_filtered.female == 0),
    'male_45_less': (data_filtered.age < 45) & (data_filtered.female == 0),
    'female_45_or_more': (data_filtered.age >= 45) & (data_filtered.female == 1),
    'female_45_less': (data_filtered.age < 45) & (data_filtered.female == 1),
}

sample_segments = {
    segment_name: segment_rows.sum() for segment_name, segment_rows in filters.items()
}
print(sample_segments)

total_sample = sum(sample_segments.values())
print(f'Sample size: {total_sample}')

weights = {
    segment_name: populations[segment_name] * total_sample / (segment_size * total_pop)
    for segment_name, segment_size in sample_segments.items()
}
print(weights)

from biogeme.biogeme import BIOGEME
from biogeme.expressions import Beta, Variable, log, exp

from biogeme import models

for segment_name, segment_rows in filters.items():
    data_filtered.loc[segment_rows, 'weight'] = weights[segment_name]


prob_walk = models.logit(V, None, 1)
prob_cycle = models.logit(V, None, 2)
prob_pt = models.logit(V, None, 3)
prob_car = models.logit(V, None, 4)


weight = Variable('weight')
simulate = {
    'weight': weight,
    'Prob. pt': prob_pt,
    'Prob. car': prob_car,
    'Prob. walk': prob_walk,
    'Prob. cycle': prob_cycle,
    'Revenues PT': prob_pt * cost_transit,
}


{'male_45_or_more': np.int64(896), 'male_45_less': np.int64(1442), 'female_45_or_more': np.int64(984), 'female_45_less': np.int64(1678)}
Sample size: 5000
{'male_45_or_more': np.float64(0.8877139043468962), 'male_45_less': np.float64(1.1715720875375348), 'female_45_or_more': np.float64(0.890816074423909), 'female_45_less': np.float64(0.9765425353056789)}


# Increased Fuel Cost

In [7]:
data_filtered_increasedFuel = data_filtered.copy(deep=True)
data_filtered_increasedFuel["cost_driving"] += 1.5
database = db.Database('london', data_filtered_increasedFuel)

biosim = BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_m4.get_beta_values())
display(simulated_values)

simulated_values['Weighted pt'] = (
    simulated_values['weight'] * simulated_values['Prob. pt']
)

simulated_values['Weighted car'] = (
    simulated_values['weight'] * simulated_values['Prob. car']
)

simulated_values['Weighted walk'] = (
    simulated_values['weight'] * simulated_values['Prob. walk']
)
simulated_values['Weighted cycle'] = (
    simulated_values['weight'] * simulated_values['Prob. cycle']
)

market_share_pt = simulated_values['Weighted pt'].mean()
print(f'Market share for pt: {100*market_share_pt:.1f}%')

market_share_car = simulated_values['Weighted car'].mean()
print(f'Market share for car: {100*market_share_car:.1f}%')

market_share_walk = simulated_values['Weighted walk'].mean()
print(f'Market share for walk: {100*market_share_walk:.1f}%')

market_share_cycle = simulated_values['Weighted cycle'].mean()
print(f'Market share for cycling: {100*market_share_cycle:.1f}%')

results_m4.bootstrap_samples = 100
results_bootstrapping = model_4.estimate(run_bootstrap=True)

betas = model_4.free_beta_names
b = results_bootstrapping.get_betas_for_sensitivity_analysis(betas)
left, right = biosim.confidence_intervals(b, 0.9)

# Revenues are calculated using the weighted mean of the individual quantities
simulated_values['Weighted revenues PT'] = (
    simulated_values['weight'] * simulated_values['Revenues PT']
)
      
display(left)

display(right)

,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle,Revenues PT
0,0.887714,0.472115,0.492866,2.574149e-03,0.032445,0.000000
1,0.976543,0.183882,0.526252,2.090353e-01,0.080830,0.551647
2,0.890816,0.229371,0.747656,1.638070e-06,0.022972,0.688112
3,0.887714,0.495363,0.447255,4.016074e-03,0.053366,0.000000
4,0.890816,0.244146,0.198551,5.249994e-01,0.032304,0.000000
...,...,...,...,...,...,...
4995,0.887714,0.443231,0.495879,9.397739e-03,0.051493,0.664846
4996,1.171572,0.508675,0.370619,3.144747e-02,0.089259,0.763013
4997,0.976543,0.263838,0.258289,4.154021e-01,0.062471,0.395757
4998,0.976543,0.344711,0.632831,7.099433e-05,0.022387,0.517067


Market share for pt: 41.5%
Market share for car: 33.7%
Market share for walk: 20.6%
Market share for cycling: 4.2%


100%|██████████| 100/100 [01:16<00:00,  1.31it/s]


,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle,Revenues PT
0,0.887714,0.424866,0.439081,1.279039e-03,0.024772,0.000000
1,0.976543,0.162580,0.495840,1.814084e-01,0.067172,0.487741
2,0.890816,0.207772,0.724422,3.617923e-07,0.016466,0.623315
3,0.887714,0.446160,0.401457,2.151975e-03,0.044123,0.000000
4,0.890816,0.215347,0.174704,4.951852e-01,0.026086,0.000000
...,...,...,...,...,...,...
4995,0.887714,0.417654,0.469626,5.544536e-03,0.043136,0.626481
4996,1.171572,0.480833,0.338594,2.036175e-02,0.077725,0.721250
4997,0.976543,0.244264,0.233847,3.872583e-01,0.051459,0.366395
4998,0.976543,0.318087,0.605907,2.374551e-05,0.018489,0.477130


,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle,Revenues PT
0,0.887714,0.521413,0.540887,5.125617e-03,0.041545,0.000000
1,0.976543,0.203551,0.563663,2.373707e-01,0.096571,0.610653
2,0.890816,0.250814,0.770831,6.385027e-06,0.033210,0.752442
3,0.887714,0.541263,0.498135,6.944179e-03,0.063151,0.000000
4,0.890816,0.272340,0.224972,5.570171e-01,0.040334,0.000000
...,...,...,...,...,...,...
4995,0.887714,0.466915,0.521960,1.480864e-02,0.060835,0.700372
4996,1.171572,0.535519,0.398898,4.605951e-02,0.102859,0.803278
4997,0.976543,0.276033,0.279141,4.427538e-01,0.073819,0.414049
4998,0.976543,0.370840,0.657814,1.808440e-04,0.027663,0.556260


In [8]:
# Calculate weighted probabilities
left['Weighted pt'] = left['weight'] * left['Prob. pt']
left['Weighted car'] = left['weight'] * left['Prob. car']
left['Weighted walk'] = left['weight'] * left['Prob. walk']
left['Weighted cycle'] = left['weight'] * left['Prob. cycle']

right['Weighted pt'] = right['weight'] * right['Prob. pt']
right['Weighted car'] = right['weight'] * right['Prob. car']
right['Weighted walk'] = right['weight'] * right['Prob. walk']
right['Weighted cycle'] = right['weight'] * right['Prob. cycle']

# Calculate mean market shares
market_share_pt = simulated_values['Weighted pt'].mean()
market_share_car = simulated_values['Weighted car'].mean()
market_share_walk = simulated_values['Weighted walk'].mean()
market_share_cycle = simulated_values['Weighted cycle'].mean()

# Calculate confidence intervals
left_market_share_pt = left['Weighted pt'].mean()
right_market_share_pt = right['Weighted pt'].mean()

left_market_share_car = left['Weighted car'].mean()
right_market_share_car = right['Weighted car'].mean()

left_market_share_walk = left['Weighted walk'].mean()
right_market_share_walk = right['Weighted walk'].mean()

left_market_share_cycle = left['Weighted cycle'].mean()
right_market_share_cycle = right['Weighted cycle'].mean()

# Print market shares and confidence intervals
print(f"Market share for pt: {100 * market_share_pt:.1f}% "
      f"CI: [{100 * left_market_share_pt:.1f}%-{100 * right_market_share_pt:.1f}%]")

print(f"Market share for car: {100 * market_share_car:.1f}% "
      f"CI: [{100 * left_market_share_car:.1f}%-{100 * right_market_share_car:.1f}%]")

print(f"Market share for walk: {100 * market_share_walk:.1f}% "
      f"CI: [{100 * left_market_share_walk:.1f}%-{100 * right_market_share_walk:.1f}%]")

print(f"Market share for cycling: {100 * market_share_cycle:.1f}% "
      f"CI: [{100 * left_market_share_cycle:.1f}%-{100 * right_market_share_cycle:.1f}%]")

# Revenues are calculated using the weighted mean of the individual quantities
simulated_values['Weighted revenues PT'] = (
    simulated_values['weight'] * simulated_values['Revenues PT']
)

left['Weighted revenues PT'] = left['weight'] * left['Revenues PT']
right['Weighted revenues PT'] = right['weight'] * right['Revenues PT']

revenues_pt = simulated_values['Weighted revenues PT'].mean()
revenues_pt_left = left['Weighted revenues PT'].mean()
revenues_pt_right = right['Weighted revenues PT'].mean()
    
print(f"Revenue per person public transport: {revenues_pt:.3f} "
      f"CI: [{revenues_pt_left:.3f}-{revenues_pt_right:.3f}]")

print(f"Total Revenue public transport: {revenues_pt*total_pop:.3f} "
      f"CI: [{revenues_pt_left*total_pop:.3f}-{revenues_pt_right*total_pop:.3f}]")

Market share for pt: 41.5% CI: [38.4%-44.3%]
Market share for car: 33.7% CI: [30.7%-36.8%]
Market share for walk: 20.6% CI: [19.2%-22.3%]
Market share for cycling: 4.2% CI: [3.4%-5.1%]
Revenue per person public transport: 0.781 CI: [0.736-0.821]
Total Revenue public transport: 6771003.763 CI: [6379388.259-7120596.017]


# Decreased Transportation Cost

In [9]:
data_filtered_loweredTransit = data_filtered.copy(deep=True)
data_filtered_loweredTransit["cost_transit"] *= 0.8
database = db.Database('london', data_filtered_loweredTransit)

biosim = BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_m4.get_beta_values())
display(simulated_values)

simulated_values['Weighted pt'] = (
    simulated_values['weight'] * simulated_values['Prob. pt']
)

simulated_values['Weighted car'] = (
    simulated_values['weight'] * simulated_values['Prob. car']
)

simulated_values['Weighted walk'] = (
    simulated_values['weight'] * simulated_values['Prob. walk']
)
simulated_values['Weighted cycle'] = (
    simulated_values['weight'] * simulated_values['Prob. cycle']
)


market_share_pt = simulated_values['Weighted pt'].mean()
print(f'Market share for pt: {100*market_share_pt:.1f}%')

market_share_car = simulated_values['Weighted car'].mean()
print(f'Market share for car: {100*market_share_car:.1f}%')

market_share_walk = simulated_values['Weighted walk'].mean()
print(f'Market share for walk: {100*market_share_walk:.1f}%')

market_share_cycle = simulated_values['Weighted cycle'].mean()
print(f'Market share for cycling: {100*market_share_cycle:.1f}%')

results_m4.bootstrap_samples = 100
results_bootstrapping = model_4.estimate(run_bootstrap=True)

betas = model_4.free_beta_names
b = results_bootstrapping.get_betas_for_sensitivity_analysis(betas)
left, right = biosim.confidence_intervals(b, 0.9)
    
display(left)

display(right)

,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle,Revenues PT
0,0.887714,0.321487,0.654667,1.752869e-03,0.022093,0.000000
1,0.976543,0.118466,0.688189,1.394303e-01,0.053915,0.284319
2,0.890816,0.168453,0.814079,1.245536e-06,0.017467,0.404287
3,0.887714,0.364979,0.592742,2.959008e-03,0.039319,0.000000
4,0.890816,0.205041,0.326919,4.409106e-01,0.027130,0.000000
...,...,...,...,...,...,...
4995,0.887714,0.330307,0.630033,6.121186e-03,0.033540,0.396368
4996,1.171572,0.414526,0.499500,2.239866e-02,0.063575,0.497432
4997,0.976543,0.237044,0.387700,3.262006e-01,0.049056,0.284452
4998,0.976543,0.273838,0.710568,4.929325e-05,0.015544,0.328606


Market share for pt: 36.6%
Market share for car: 43.0%
Market share for walk: 17.2%
Market share for cycling: 3.2%


100%|██████████| 100/100 [00:59<00:00,  1.68it/s]


,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle,Revenues PT
0,0.887714,0.294965,0.623701,6.597082e-04,0.017238,0.000000
1,0.976543,0.104918,0.663949,1.169677e-01,0.045258,0.251804
2,0.890816,0.148429,0.795822,2.541186e-07,0.011284,0.356230
3,0.887714,0.340730,0.563736,1.453436e-03,0.032465,0.000000
4,0.890816,0.188655,0.309035,4.133874e-01,0.021995,0.000000
...,...,...,...,...,...,...
4995,0.887714,0.313453,0.613071,3.490199e-03,0.028936,0.376143
4996,1.171572,0.394400,0.473877,1.449498e-02,0.055247,0.473280
4997,0.976543,0.225794,0.371608,3.017097e-01,0.041102,0.270952
4998,0.976543,0.257577,0.690887,1.868258e-05,0.011278,0.309093


,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle,Revenues PT
0,0.887714,0.354433,0.682940,3.322455e-03,0.027080,0.000000
1,0.976543,0.130581,0.712734,1.555040e-01,0.063884,0.313395
2,0.890816,0.186729,0.836066,3.623813e-06,0.023544,0.448151
3,0.887714,0.395308,0.615864,4.992158e-03,0.046475,0.000000
4,0.890816,0.226535,0.342368,4.687810e-01,0.033364,0.000000
...,...,...,...,...,...,...
4995,0.887714,0.348738,0.647471,8.808438e-03,0.039299,0.418486
4996,1.171572,0.437363,0.520712,2.884730e-02,0.074689,0.524836
4997,0.976543,0.249959,0.405174,3.476317e-01,0.057982,0.299951
4998,0.976543,0.292199,0.727582,9.157709e-05,0.018971,0.350639


In [10]:
# Calculate weighted probabilities
left['Weighted pt'] = left['weight'] * left['Prob. pt']
left['Weighted car'] = left['weight'] * left['Prob. car']
left['Weighted walk'] = left['weight'] * left['Prob. walk']
left['Weighted cycle'] = left['weight'] * left['Prob. cycle']

right['Weighted pt'] = right['weight'] * right['Prob. pt']
right['Weighted car'] = right['weight'] * right['Prob. car']
right['Weighted walk'] = right['weight'] * right['Prob. walk']
right['Weighted cycle'] = right['weight'] * right['Prob. cycle']

# Calculate mean market shares
market_share_pt = simulated_values['Weighted pt'].mean()
market_share_car = simulated_values['Weighted car'].mean()
market_share_walk = simulated_values['Weighted walk'].mean()
market_share_cycle = simulated_values['Weighted cycle'].mean()

# Calculate confidence intervals
left_market_share_pt = left['Weighted pt'].mean()
right_market_share_pt = right['Weighted pt'].mean()

left_market_share_car = left['Weighted car'].mean()
right_market_share_car = right['Weighted car'].mean()

left_market_share_walk = left['Weighted walk'].mean()
right_market_share_walk = right['Weighted walk'].mean()

left_market_share_cycle = left['Weighted cycle'].mean()
right_market_share_cycle = right['Weighted cycle'].mean()

# Print market shares and confidence intervals
print(f"Market share for pt: {100 * market_share_pt:.1f}% "
      f"CI: [{100 * left_market_share_pt:.1f}%-{100 * right_market_share_pt:.1f}%]")

print(f"Market share for car: {100 * market_share_car:.1f}% "
      f"CI: [{100 * left_market_share_car:.1f}%-{100 * right_market_share_car:.1f}%]")

print(f"Market share for walk: {100 * market_share_walk:.1f}% "
      f"CI: [{100 * left_market_share_walk:.1f}%-{100 * right_market_share_walk:.1f}%]")

print(f"Market share for cycling: {100 * market_share_cycle:.1f}% "
      f"CI: [{100 * left_market_share_cycle:.1f}%-{100 * right_market_share_cycle:.1f}%]")

# Revenues are calculated using the weighted mean of the individual quantities
simulated_values['Weighted revenues PT'] = (
    simulated_values['weight'] * simulated_values['Revenues PT']
)

left['Weighted revenues PT'] = left['weight'] * left['Revenues PT']
right['Weighted revenues PT'] = right['weight'] * right['Revenues PT']

revenues_pt = simulated_values['Weighted revenues PT'].mean()
revenues_pt_left = left['Weighted revenues PT'].mean()
revenues_pt_right = right['Weighted revenues PT'].mean()
    
print(f"Revenue per person public transport: {revenues_pt:.3f} "
      f"CI: [{revenues_pt_left:.3f}-{revenues_pt_right:.3f}]")

print(f"Total Revenue public transport: {revenues_pt*total_pop:.3f} "
      f"CI: [{revenues_pt_left*total_pop:.3f}-{revenues_pt_right*total_pop:.3f}]")

Market share for pt: 36.6% CI: [34.3%-39.0%]
Market share for car: 43.0% CI: [40.4%-45.4%]
Market share for walk: 17.2% CI: [16.0%-18.6%]
Market share for cycling: 3.2% CI: [2.6%-4.0%]
Revenue per person public transport: 0.581 CI: [0.549-0.611]
Total Revenue public transport: 5040909.947 CI: [4762947.918-5296684.939]


# Highest Total Revenue

In [11]:
database = db.Database('london', data_filtered)

biosim = BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_m4.get_beta_values())
display(simulated_values)

simulated_values['Weighted pt'] = (
    simulated_values['weight'] * simulated_values['Prob. pt']
)

simulated_values['Weighted car'] = (
    simulated_values['weight'] * simulated_values['Prob. car']
)

simulated_values['Weighted walk'] = (
    simulated_values['weight'] * simulated_values['Prob. walk']
)
simulated_values['Weighted cycle'] = (
    simulated_values['weight'] * simulated_values['Prob. cycle']
)

market_share_pt = simulated_values['Weighted pt'].mean()
print(f'Market share for pt: {100*market_share_pt:.1f}%')

market_share_car = simulated_values['Weighted car'].mean()
print(f'Market share for car: {100*market_share_car:.1f}%')

market_share_walk = simulated_values['Weighted walk'].mean()
print(f'Market share for walk: {100*market_share_walk:.1f}%')

market_share_cycle = simulated_values['Weighted cycle'].mean()
print(f'Market share for cycling: {100*market_share_cycle:.1f}%')

results_m4.bootstrap_samples = 100
results_bootstrapping = model_4.estimate(run_bootstrap=True)

betas = model_4.free_beta_names
b = results_bootstrapping.get_betas_for_sensitivity_analysis(betas)
left, right = biosim.confidence_intervals(b, 0.9)

# Revenues are calculated using the weighted mean of the individual quantities
simulated_values['Weighted revenues PT'] = (
    simulated_values['weight'] * simulated_values['Revenues PT']
)
      
display(left)

display(right)

,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle,Revenues PT
0,0.887714,0.321487,0.654667,1.752869e-03,0.022093,0.000000
1,0.976543,0.122141,0.685319,1.388490e-01,0.053690,0.366424
2,0.890816,0.173374,0.809261,1.238165e-06,0.017364,0.520122
3,0.887714,0.364979,0.592742,2.959008e-03,0.039319,0.000000
4,0.890816,0.205041,0.326919,4.409106e-01,0.027130,0.000000
...,...,...,...,...,...,...
4995,0.887714,0.301231,0.657386,6.386945e-03,0.034996,0.451846
4996,1.171572,0.382269,0.527020,2.363274e-02,0.067078,0.573404
4997,0.976543,0.213560,0.399633,3.362411e-01,0.050566,0.320339
4998,0.976543,0.247894,0.735956,5.105444e-05,0.016099,0.371840


Market share for pt: 35.7%
Market share for car: 43.5%
Market share for walk: 17.5%
Market share for cycling: 3.3%


100%|██████████| 100/100 [01:05<00:00,  1.52it/s]


,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle,Revenues PT
0,0.887714,0.298359,0.624249,8.162307e-04,0.017344,0.000000
1,0.976543,0.104614,0.668224,1.229207e-01,0.044418,0.313843
2,0.890816,0.155181,0.793743,3.264616e-07,0.012793,0.465543
3,0.887714,0.342800,0.562920,1.572700e-03,0.031841,0.000000
4,0.890816,0.186186,0.306212,4.112358e-01,0.021859,0.000000
...,...,...,...,...,...,...
4995,0.887714,0.283789,0.638725,4.129027e-03,0.030292,0.425684
4996,1.171572,0.356960,0.501319,1.663179e-02,0.057729,0.535439
4997,0.976543,0.198351,0.385515,3.169230e-01,0.041302,0.297527
4998,0.976543,0.226807,0.719128,2.140344e-05,0.011970,0.340210


,weight,Prob. pt,Prob. car,Prob. walk,Prob. cycle,Revenues PT
0,0.887714,0.352183,0.677947,3.456817e-03,0.027556,0.000000
1,0.976543,0.134822,0.711278,1.582858e-01,0.062551,0.404466
2,0.890816,0.189105,0.831390,3.991791e-06,0.023077,0.567315
3,0.887714,0.393760,0.615505,5.066375e-03,0.045474,0.000000
4,0.890816,0.227371,0.344857,4.718854e-01,0.033916,0.000000
...,...,...,...,...,...,...
4995,0.887714,0.318859,0.676422,9.362330e-03,0.040449,0.478288
4996,1.171572,0.407373,0.553818,3.137005e-02,0.075227,0.611060
4997,0.976543,0.226779,0.416454,3.621281e-01,0.059107,0.340168
4998,0.976543,0.263260,0.758401,1.005173e-04,0.019030,0.394890


In [12]:
# Calculate weighted probabilities
left['Weighted pt'] = left['weight'] * left['Prob. pt']
left['Weighted car'] = left['weight'] * left['Prob. car']
left['Weighted walk'] = left['weight'] * left['Prob. walk']
left['Weighted cycle'] = left['weight'] * left['Prob. cycle']

right['Weighted pt'] = right['weight'] * right['Prob. pt']
right['Weighted car'] = right['weight'] * right['Prob. car']
right['Weighted walk'] = right['weight'] * right['Prob. walk']
right['Weighted cycle'] = right['weight'] * right['Prob. cycle']

# Calculate mean market shares
market_share_pt = simulated_values['Weighted pt'].mean()
market_share_car = simulated_values['Weighted car'].mean()
market_share_walk = simulated_values['Weighted walk'].mean()
market_share_cycle = simulated_values['Weighted cycle'].mean()

# Calculate confidence intervals
left_market_share_pt = left['Weighted pt'].mean()
right_market_share_pt = right['Weighted pt'].mean()

left_market_share_car = left['Weighted car'].mean()
right_market_share_car = right['Weighted car'].mean()

left_market_share_walk = left['Weighted walk'].mean()
right_market_share_walk = right['Weighted walk'].mean()

left_market_share_cycle = left['Weighted cycle'].mean()
right_market_share_cycle = right['Weighted cycle'].mean()

# Print market shares and confidence intervals
print(f"Market share for pt: {100 * market_share_pt:.1f}% "
      f"CI: [{100 * left_market_share_pt:.1f}%-{100 * right_market_share_pt:.1f}%]")

print(f"Market share for car: {100 * market_share_car:.1f}% "
      f"CI: [{100 * left_market_share_car:.1f}%-{100 * right_market_share_car:.1f}%]")

print(f"Market share for walk: {100 * market_share_walk:.1f}% "
      f"CI: [{100 * left_market_share_walk:.1f}%-{100 * right_market_share_walk:.1f}%]")

print(f"Market share for cycling: {100 * market_share_cycle:.1f}% "
      f"CI: [{100 * left_market_share_cycle:.1f}%-{100 * right_market_share_cycle:.1f}%]")

# Revenues are calculated using the weighted mean of the individual quantities
simulated_values['Weighted revenues PT'] = (
    simulated_values['weight'] * simulated_values['Revenues PT']
)

left['Weighted revenues PT'] = left['weight'] * left['Revenues PT']
right['Weighted revenues PT'] = right['weight'] * right['Revenues PT']

revenues_pt = simulated_values['Weighted revenues PT'].mean()
revenues_pt_left = left['Weighted revenues PT'].mean()
revenues_pt_right = right['Weighted revenues PT'].mean()
    
print(f"Revenue per person public transport: {revenues_pt:.3f} "
      f"CI: [{revenues_pt_left:.3f}-{revenues_pt_right:.3f}]")

print(f"Total Revenue public transport: {revenues_pt*total_pop:.3f} "
      f"CI: [{revenues_pt_left*total_pop:.3f}-{revenues_pt_right*total_pop:.3f}]")

Market share for pt: 35.7% CI: [33.3%-38.1%]
Market share for car: 43.5% CI: [41.1%-46.1%]
Market share for walk: 17.5% CI: [16.2%-18.8%]
Market share for cycling: 3.3% CI: [2.6%-3.9%]
Revenue per person public transport: 0.714 CI: [0.673-0.751]
Total Revenue public transport: 6190605.949 CI: [5830663.347-6508177.218]


# Value of Time

In [13]:
from biogeme.expressions import Derive
from biogeme.biogeme import BIOGEME
from biogeme.expressions import Beta, Variable, log, exp
from biogeme import models

database = db.Database('london', data_filtered)


for segment_name, segment_rows in filters.items():
    data_filtered.loc[segment_rows, 'weight'] = weights[segment_name]

weight = Variable('weight')
vot_pt = Derive(V_PT, 'dur_pt') / Derive(V_PT, 'cost_transit')
vot_car = Derive(V_DRIVE, 'dur_driving') / Derive(V_DRIVE, 'cost_driving')
simulate = {
    'weight': weight,
    'WTP PT time': vot_pt,
    'WTP CAR time': vot_car,
}
biosim = BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_m4.get_beta_values())

print(-simulated_values['WTP PT time'].mean())
print(-simulated_values['WTP CAR time'].mean())

Parameter ASC_CYCLE not present in the model.
Parameter B_TIME_CYCLE not present in the model.
Parameter B_TIME_WALK not present in the model.
Parameter G_TIME_CYCLE_AGE not present in the model.
Parameter G_TIME_WALK_AGE not present in the model.


8.456379977048066
-8.410365569381995


# Elasticities

In [15]:
prob_walk = models.logit(V, None, 1)
prob_cycle = models.logit(V, None, 2)
prob_pt = models.logit(V, None, 3)
prob_car = models.logit(V, None, 4)

direct_elas_car_cost = Derive(prob_car, 'cost_driving') * dur_driving / prob_car
direct_elas_pt_cost = Derive(prob_pt, 'cost_transit') * cost_transit / prob_pt

cross_elas_prob_car_pt_cost = Derive(prob_car, 'cost_transit') * cost_transit / prob_car
cross_elas_prob_walk_pt_cost = Derive(prob_walk, 'cost_transit') * cost_transit / prob_walk
cross_elas_prob_cycle_pt_cost = Derive(prob_cycle, 'cost_transit') * cost_transit / prob_cycle

cross_elas_prob_pt_car_cost = Derive(prob_pt, 'cost_driving') * cost_driving / prob_pt
cross_elas_prob_walk_car_cost = Derive(prob_walk, 'cost_driving') * cost_driving / prob_walk
cross_elas_prob_cycle_car_cost = Derive(prob_cycle, 'cost_driving') * cost_driving / prob_cycle

simulate = {
    'weight': weight,
    'Prob. pt': prob_pt,
    'Prob. car': prob_car,
    'Prob. walk': prob_walk,
    'Prob. cycle': prob_cycle,
    'WTP PT time': vot_pt,
    'WTP CAR time': vot_car,
    'direct_elas_pt_cost': direct_elas_pt_cost,
    'direct_elas_car_cost': direct_elas_car_cost,
    'cross_elas_prob_car_pt_cost': cross_elas_prob_car_pt_cost,
    'cross_elas_prob_walk_pt_cost': cross_elas_prob_walk_pt_cost,
    'cross_elas_prob_cycle_pt_cost': cross_elas_prob_cycle_pt_cost,
    'cross_elas_prob_pt_car_cost': cross_elas_prob_pt_car_cost,
    'cross_elas_prob_walk_car_cost': cross_elas_prob_walk_car_cost,
    'cross_elas_prob_cycle_car_cost': cross_elas_prob_cycle_car_cost
}

biosim = BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_m4.get_beta_values())

simulated_values['numerator_pt_cost'] = (
    simulated_values['weight']
    * simulated_values['Prob. pt']
    * simulated_values['direct_elas_pt_cost']
)

simulated_values['denominator_pt'] = (
    simulated_values['weight'] * simulated_values['Prob. pt']
)

agg_elast_pt_cost = (
    simulated_values['numerator_pt_cost'].sum()
    / simulated_values['denominator_pt'].sum()
)

print(f'Aggregate PT elasticity wrt cost: {agg_elast_pt_cost:.3g}')

simulated_values['numerator_car_cost'] = (
    simulated_values['weight']
    * simulated_values['Prob. car']
    * simulated_values['direct_elas_car_cost']
)

simulated_values['denominator_car'] = (
    simulated_values['weight'] * simulated_values['Prob. car']
)

agg_elast_car_cost = (
    simulated_values['numerator_car_cost'].sum()
    / simulated_values['denominator_car'].sum()
)

print(f'Aggregate CAR elasticity wrt cost: {agg_elast_car_cost:.3g}')

simulated_values['numerator_cross_car_pt_cost'] = (
    simulated_values['weight']
    * simulated_values['Prob. car']
    * simulated_values['cross_elas_prob_car_pt_cost']
)

agg_elast_pt_cost = (
    simulated_values['numerator_cross_car_pt_cost'].sum()
    / simulated_values['denominator_car'].sum()
)

print(f'Cross CAR elasticity wrt PT cost: {agg_elast_pt_cost:.3g}')

simulated_values['numerator_cross_walk_pt_cost'] = (
    simulated_values['weight']
    * simulated_values['Prob. walk']
    * simulated_values['cross_elas_prob_walk_pt_cost']
)

simulated_values['denominator_walk'] = (
    simulated_values['weight'] * simulated_values['Prob. walk']
)

agg_elast_cross_walk_pt_cost = (
    simulated_values['numerator_cross_walk_pt_cost'].sum()
    / simulated_values['denominator_walk'].sum()
)

print(f'Cross WALK elasticity wrt PT cost: {agg_elast_cross_walk_pt_cost:.3g}')

simulated_values['numerator_cross_cycle_pt_cost'] = (
    simulated_values['weight']
    * simulated_values['Prob. cycle']
    * simulated_values['cross_elas_prob_cycle_pt_cost']
)

simulated_values['denominator_cycle'] = (
    simulated_values['weight'] * simulated_values['Prob. cycle']
)

agg_elast_cross_cycle_pt_cost = (
    simulated_values['numerator_cross_cycle_pt_cost'].sum()
    / simulated_values['denominator_cycle'].sum()
)

print(f'Cross CYCLE elasticity wrt PT cost: {agg_elast_cross_cycle_pt_cost:.3g}')


# NOW FOR CAR TIME

simulated_values['numerator_cross_pt_car_cost'] = (
    simulated_values['weight']
    * simulated_values['Prob. pt']
    * simulated_values['cross_elas_prob_pt_car_cost']
)

agg_elast_cross_pt_car_cost = (
    simulated_values['numerator_cross_pt_car_cost'].sum()
    / simulated_values['denominator_pt'].sum()
)

print(f'Cross PT elasticity wrt CAR cost: {agg_elast_cross_pt_car_cost:.3g}')

simulated_values['numerator_cross_walk_car_cost'] = (
    simulated_values['weight']
    * simulated_values['Prob. walk']
    * simulated_values['cross_elas_prob_walk_car_cost']
)

agg_elast_cross_walk_car_cost = (
    simulated_values['numerator_cross_walk_car_cost'].sum()
    / simulated_values['denominator_walk'].sum()
)

print(f'Cross WALK elasticity wrt CAR cost: {agg_elast_cross_walk_car_cost:.3g}')

simulated_values['numerator_cross_cycle_car_cost'] = (
    simulated_values['weight']
    * simulated_values['Prob. walk']
    * simulated_values['cross_elas_prob_walk_car_cost']
)

agg_elast_cross_walk_car_cost = (
    simulated_values['numerator_cross_walk_car_cost'].sum()
    / simulated_values['denominator_walk'].sum()
)

print(f'Cross WALK elasticity wrt CAR cost: {agg_elast_cross_walk_car_cost:.3g}')

Aggregate PT elasticity wrt cost: -0.114
Aggregate CAR elasticity wrt cost: -0.0384
Cross CAR elasticity wrt PT cost: 0.065
Cross WALK elasticity wrt PT cost: 0.0564
Cross CYCLE elasticity wrt PT cost: 0.0757
Cross PT elasticity wrt CAR cost: 0.143
Cross WALK elasticity wrt CAR cost: 0.0464
Cross WALK elasticity wrt CAR cost: 0.0464
